# Contexto Personalizado - Academia de la Flota Estelar

## Objetivo
Aprender cómo **extraer el perfil de un usuario desde su historial conversacional** e inyectarlo en el thread para crear experiencias personalizadas.

### Escenario
Eres el **Asesor de la Academia de la Flota Estelar**. Un cadete vulcano solicita ayuda para elegir un proyecto de misión. ¿Le darás la misma recomendación genérica a todos, o adaptarás tus respuestas a su perfil, raza y objetivos?

**Diferencia clave:** Cómo el contexto personalizado transforma respuestas genéricas en respuestas estratégicamente adaptadas.

In [2]:
import os
import asyncio
import json
import re
import textwrap
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv

# Cargar configuración
load_dotenv()
base_url = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
model_id = os.getenv("AZURE_OPENAI_DEPLOYMENT")

print("=" * 80)
print("✅ Cliente OpenAI configurado")
print(f"📍 Modelo: {model_id}")
print("=" * 80)

✅ Cliente OpenAI configurado
📍 Modelo: gpt-4


## Paso 1: Construir Conversación y Extraer Perfil

Comenzamos recopilando información del cadete a través de una conversación natural. Luego extraeremos su perfil de forma estructurada.

In [3]:
# Crear cliente y agente
client = OpenAIChatClient(
    base_url=base_url,
    api_key=api_key,
    model_id=model_id
)

agent = client.create_agent(
    name="AsesorAcademiaEstelar",
    instructions="Eres un asesor de la Academia de la Flota Estelar que proporciona recomendaciones personalizadas."
)

print("\n✅ Agente 'AsesorAcademiaEstelar' creado\n")

# Crear thread y construir conversación
thread = agent.get_new_thread()
registro_conversacion = []

mensajes_cadete = [
    "Saludos, soy el Alférez T'Pral de Vulcano",
    "Tengo experiencia intermedia en programación de sistemas",
    "Estoy muy interesado en automatización de navegación y IA",
    "Quiero aprender a construir agentes inteligentes",
    "Prefiero explicaciones técnicas y método lógico"
]

print("📚 Construyendo perfil conversacional del cadete T'Pral...\n")
print("-" * 80)

for msg in mensajes_cadete:
    result = await agent.run(msg, thread=thread)
    registro_conversacion.append({
        "cadete": msg,
        "asesor": result.text
    })
    print(f"Cadete: {msg}")
    print(f"Asesor: {result.text}...\n")

print("-" * 80)
print(f"✅ Historial construido: {len(registro_conversacion)} intercambios\n")


✅ Agente 'AsesorAcademiaEstelar' creado

📚 Construyendo perfil conversacional del cadete T'Pral...

--------------------------------------------------------------------------------
Cadete: Saludos, soy el Alférez T'Pral de Vulcano
Asesor: Saludos, Alférez T’Pral. Es un honor asistirle. Como vulcana, su enfoque lógico y disciplina constituyen fortalezas notables dentro de la Flota Estelar. ¿En qué área desea recomendaciones hoy? Por ejemplo:

- Orientación en la elección de especialidad
- Estrategias para el dominio de rutinas de meditación durante misiones exigentes
- Recomendaciones para interacción eficiente con tripulaciones multi-especie
- Consejos para el ascenso en el rango

Por favor, indique sus intereses o necesidades específicas para que pueda brindarle una asesoría precisa y lógica....

Cadete: Tengo experiencia intermedia en programación de sistemas
Asesor: Excelente perfil, Alférez T’Pral. Sus competencias intermedias en programación de sistemas la sitúan en una posición 

### Extrayendo el Perfil Estructurado

Ahora le pedimos al agente que analice la conversación y extraiga un perfil JSON estructurado con los datos clave del cadete.

In [4]:
# Convertir conversación a texto
def intercambios_a_texto(intercambios):
    texto = ""
    for intercambio in intercambios:
        texto += f"Cadete: {intercambio['cadete']}\n"
        texto += f"Asesor: {intercambio['asesor']}\n\n"
    return texto

texto_conversacion = intercambios_a_texto(registro_conversacion)

# Solicitar extracción de perfil
prompt_extraccion = f"""Analiza esta conversación y extrae los datos del cadete en JSON:

{texto_conversacion}

Extrae SÓLO este JSON (sin texto adicional):
{{
    "nombre": "nombre",
    "raza": "raza",
    "nivel_habilidad": "principiante/intermedio/avanzado",
    "intereses": ["lista", "de", "intereses"],
    "objetivos": ["lista", "de", "objetivos"],
    "estilo_comunicacion": "preferencia"
}}"""

print("🔍 Extrayendo perfil del cadete...\n")
resultado_perfil = await agent.run(prompt_extraccion)
perfil_json = resultado_perfil.text

# Parsear JSON
try:
    coincidencia_json = re.search(r'\{[^}]+\}', perfil_json, re.DOTALL)
    if coincidencia_json:
        perfil_json = coincidencia_json.group(0)
    
    perfil_cadete = json.loads(perfil_json)
except:
    perfil_cadete = {
        "nombre": "Alférez T'Pral",
        "raza": "Vulcano",
        "nivel_habilidad": "intermedio",
        "intereses": ["automatización de navegación", "IA"],
        "objetivos": ["construir agentes inteligentes"],
        "estilo_comunicacion": "técnico lógico"
    }

print("=" * 80)
print("📊 PERFIL EXTRAÍDO DEL CADETE")
print("=" * 80)
print(f"Nombre: {perfil_cadete.get('nombre', 'T\'Pral')}")
print(f"Raza: {perfil_cadete.get('raza', 'Vulcano')}")
print(f"Nivel: {perfil_cadete.get('nivel_habilidad', 'intermedio')}")
print(f"Intereses: {', '.join(perfil_cadete.get('intereses', []))}")
print(f"Objetivos: {', '.join(perfil_cadete.get('objetivos', []))}")
print(f"Estilo: {perfil_cadete.get('estilo_comunicacion', 'técnico')}")
print("=" * 80 + "\n")

🔍 Extrayendo perfil del cadete...

📊 PERFIL EXTRAÍDO DEL CADETE
Nombre: T'Pral
Raza: Vulcano
Nivel: intermedio
Intereses: programación de sistemas, automatización de navegación, Inteligencia Artificial, construcción de agentes inteligentes
Objetivos: profundizar habilidades técnicas, especializarse en IA y navegación autónoma, aprender a construir agentes inteligentes para navegación estelar
Estilo: explicaciones técnicas y método lógico



## Paso 2: Comparar Respuestas - Genérica vs Personalizada

Ahora haremos la **pregunta clave** en dos escenarios:

1. **Sin contexto personalizado** → Respuesta genérica
2. **Con perfil inyectado** → Respuesta personalizada

Observa cómo el mismo agente adapta su recomendación basándose en el contexto.

In [5]:
consulta_prueba = """Recomiéndame un proyecto de misión para desarrollar habilidades en agentes inteligentes 
en un par de parrafos"""

print("\n" + "=" * 80)
print("🧪 PRUEBA 1: Respuesta GENÉRICA (Sin Contexto Personalizado)")
print("=" * 80)
print(f"Pregunta: {consulta_prueba}\n")

# Thread sin contexto personalizado
thread_generico = agent.get_new_thread()
resultado_generico = await agent.run(consulta_prueba, thread=thread_generico)

print("📝 RESPUESTA GENÉRICA:")
print("-" * 80)
# Formatear texto para que se ajuste al ancho de 78 caracteres
respuesta_formateada = textwrap.fill(resultado_generico.text, width=78)
print(respuesta_formateada)
print("-" * 80)


🧪 PRUEBA 1: Respuesta GENÉRICA (Sin Contexto Personalizado)
Pregunta: Recomiéndame un proyecto de misión para desarrollar habilidades en agentes inteligentes 
en un par de parrafos

📝 RESPUESTA GENÉRICA:
--------------------------------------------------------------------------------
Recomiendo un proyecto de simulación de “Primer Contacto” en el que los
cadetes diseñen, desplieguen y gestionen agentes inteligentes autónomos
encargados de interactuar con una civilización previamente desconocida. Los
cadetes deberán programar los agentes para interpretar señales desconocidas,
analizar patrones lingüísticos y culturales, y establecer canales de
comunicación efectivos, todo mientras respetan la Primera Directiva. Este
entorno de simulación puede incluir eventos aleatorios y dilemas éticos para
reforzar la toma de decisiones adaptativa y responsable.  Durante la misión,
los cadetes serán responsables de monitorizar los comportamientos de los
agentes, ajustar sus algoritmos de aprendizaje 

In [6]:
print("\n" + "=" * 80)
print("🧪 PRUEBA 2: Respuesta PERSONALIZADA (Con Perfil Inyectado en el Thread)")
print("=" * 80)
print(f"Pregunta: {consulta_prueba}\n")

# Thread con contexto personalizado
thread_personalizado = agent.get_new_thread()

# ⚡ PASO 1: Inyectar perfil como PRIMER mensaje en el thread
perfil_mensaje = f"""Tengo que ayudarte con un cadete específico. Por favor, ten en CUENTA este perfil en todas tus respuestas futuras:

📋 PERFIL DEL CADETE:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Nombre: {perfil_cadete.get('nombre', 'T\'Pral')}
Raza: {perfil_cadete.get('raza', 'Vulcano')}
Nivel de habilidad: {perfil_cadete.get('nivel_habilidad', 'intermedio')}
Intereses: {', '.join(perfil_cadete.get('intereses', []))}
Objetivos: {', '.join(perfil_cadete.get('objetivos', []))}
Estilo de comunicación: {perfil_cadete.get('estilo_comunicacion', 'técnica')}
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Recuerda este contexto y sé ESPECÍFICO y PERSONALIZADO en tus recomendaciones."""

print("🔄 Inyectando perfil en el thread...\n")
await agent.run(perfil_mensaje, thread=thread_personalizado)
print("✅ Perfil inyectado. Ahora el thread lo conoce.\n")

# ⚡ PASO 2: Hacer la consulta (el agente ya tiene el contexto del perfil)
print("📨 Enviando consulta (con contexto de perfil en el thread)...\n")
resultado_personalizado = await agent.run(consulta_prueba, thread=thread_personalizado)

print("📝 RESPUESTA PERSONALIZADA:")
print("-" * 80)
print(resultado_personalizado.text)
print("-" * 80)


🧪 PRUEBA 2: Respuesta PERSONALIZADA (Con Perfil Inyectado en el Thread)
Pregunta: Recomiéndame un proyecto de misión para desarrollar habilidades en agentes inteligentes 
en un par de parrafos

🔄 Inyectando perfil en el thread...

✅ Perfil inyectado. Ahora el thread lo conoce.

📨 Enviando consulta (con contexto de perfil en el thread)...

📝 RESPUESTA PERSONALIZADA:
--------------------------------------------------------------------------------
Por el perfil de T'Pral, recomiendo como proyecto de misión el diseño y simulación de un **Agente Autónomo de Diagnóstico de Anomalías de Navegación** para una nave de la Flota Estelar. El objetivo principal será construir un agente inteligente capaz de monitorear constantemente los sistemas de navegación, detectar patrones irregulares (por ejemplo, desvíos de rumbo inesperados, perturba­ciones de campo subespacial o fluctuaciones en los sensores de trayectoria) y proponer acciones correctivas de forma automática. El agente deberá incorporar mó

## 🎯 Conclusión

La personalización **NO** hace la respuesta más larga, sino más **RELEVANTE**.

### El Agente Modifica:
- **Tono** → Más lógico para vulcanos
- **Contenido** → Enfocado en IA y automatización  
- **Referencias** → Usa datos del perfil específico
- **Contexto** → Respeta objetivos y nivel del cadete

### Patrón en Producción:

1. **Construir conversación** → Registrar intercambios naturales
2. **Extraer perfil** → JSON estructurado con datos clave
3. **Inyectar contexto** → Como mensaje en el thread
4. **Resultado** → Respuestas personalizadas automáticas

### Aplicaciones Reales:

| Industria | Uso |
|-----------|-----|
| **E-commerce** | Recomendaciones por tipo de cliente |
| **Educación** | Contenido adaptado al nivel y estilo |
| **Soporte técnico** | Respuestas técnicas o simplificadas según experiencia |
| **Diplomacia** | Comunicación culturalmente apropiada |
| **RR.HH.** | Feedback personalizado según rol |

---

## 📊 Impacto del Cambio

**Sin personalización:** El usuario recibe una respuesta genérica que podría servir para cualquiera.

**Con personalización:** El usuario recibe una respuesta diseñada específicamente para él, considerando:
- Su experiencia actual
- Sus objetivos claros
- Sus preferencias de comunicación
- Su contexto cultural/raza

**Resultado:** Mayor relevancia, mayor valor, mayor satisfacción del usuario.